# Clasificación de texto

Este notebook presenta la creación de un dataset de oraciones con su respectivo autor, con el que luego se entrenan varios clasificadores usando variaciones de redes feed-forward, y distintos embeddings pre-entrenados.

## 0. Importación de Librerías

In [1]:
# Se importan las librerías necesarias para el desarrollo del proyecto
import re

import gensim
from gensim.parsing.preprocessing import STOPWORDS
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd
from sklearn.model_selection import train_test_split

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

## 1. Creación de dataset

Primero, creamos el dataset de oraciones etiquetadas según el autor.

In [2]:
# Se define una función que carga un archivo de texto y lo devuelve como un string
def load_raw_data(file_path: str) -> str:
    """
    Carga el texto crudo a partir de un archivo de texto
    
    Args:
    file_path (str): Ruta del archivo de texto.
    
    Returns:
    str: Texto crudo.
    """
    # Leer el texto crudo
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    return text

In [3]:
# Se define una función que extrae las oraciones del texto según el formato de los libros de Gutenberg
def extract_sentences(book: str) -> list[str]:
    """
    Extrae extractos de un libro asegurando que cumplan con ciertas condiciones de tamaño
    
    Args:
    book (str): Texto crudo.
    
    Returns:
    list[str]: Lista de extractos del libro.
    """
    # Separar el texto en bloques usando líneas completamente vacías como delimitadores
    # Seleccionar el tercer bloque que contiene el contenido del libro
    lines = book.split('***')[2].split('\n\n')

    # Eliminar espacios en blanco al inicio y al final de cada línea
    lines = [line.strip() for line in lines]

    # Eliminar lineas vacias
    lines = [line for line in lines if line]

    # Eliminar lineas genericas como ilustraciones o titulos de los capitulos de los libros
    lines = [line for line in lines if not (
        line.startswith('CHAPTER')) or 
        line.startswith('[Illustration]')
    ]

    # Eliminar saltos de lineas de las oraciones
    lines = [line.replace('\n', ' ') for line in lines]

    # Solo procesar lineas con mas de 150 caracteres
    lines = [line for line in lines if len(line) >= 150]

    # Separar adicionalmente por . si la oracion es muy larga
    sentences = []
    for sentence in lines:
        if len(sentence) > 250:
            sentences.extend(sent_tokenize(sentence))    # Dividir en oraciones usando NLTK
        else:
            sentences.append(sentence)

    # Solo procesar lineas con mas de 150 y menos de 250 caracteres
    sentences = [sentence for sentence in sentences if len(sentence) >= 150 and len(sentence) <= 250]

    # Eliminar espacios en blanco al inicio y al final de cada línea nuevamente
    sentences = [sentence.strip() for sentence in sentences]

    return sentences

In [4]:
# Ruta a los libros originales junto con su autor
raw_books = {
    'austen_sense-and-sensibility': {
        'file_path': 'data/raw/austen_sense-and-sensibility.txt',
        'author': 'Jane Austen',
    },
    'austen_pride-and-prejudice': {
        'file_path': 'data/raw/austen_pride-and-prejudice.txt',
        'author': 'Jane Austen',
    },
    'austen_emma': {
        'file_path': 'data/raw/austen_emma.txt',
        'author': 'Jane Austen',
    },
    'tolstoy_youth': {
        'file_path': 'data/raw/tolstoy_youth.txt',
        'author': 'Leo Tolstoy',
    },
    'tolstoy_war-and-peace': {
        'file_path': 'data/raw/tolstoy_war-and-peace.txt',
        'author': 'Leo Tolstoy',
    },
    'tolstoy_anna-karenina': {
        'file_path': 'data/raw/tolstoy_anna-karenina.txt',
        'author': 'Leo Tolstoy',
    },
    'joyce_dubliners': {
        'file_path': 'data/raw/joyce_dubliners.txt',
        'author': 'James Joyce',
    },
    'joyce_a-portrait-of-the-artist-as-a-young-man': {
        'file_path': 'data/raw/joyce_a-portrait-of-the-artist-as-a-young-man.txt',
        'author': 'James Joyce',
    },
    'joyce_ulysses': {
        'file_path': 'data/raw/joyce_ulysses.txt',
        'author': 'James Joyce',
    }
}

In [5]:
# Se crea un dataframe con las oraciones extraídas de los libros
df = pd.DataFrame(columns=['author', 'sentence'])

# Por cada libro, se extra el texto y se concatenan las oraciones en el dataframe
for book in raw_books.values():
    corpus = load_raw_data(book['file_path'])
    author = book['author']
    
    # Extraer las oraciones del texto
    sentences = extract_sentences(corpus)

    df = pd.concat([df, pd.DataFrame({'author': author, 'sentence': sentences})], ignore_index=True)

df.head()

,author,sentence
0,Jane Austen,"Their estate was large, and their residence wa..."
1,Jane Austen,The late owner of this estate was a single man...
2,Jane Austen,"The son, a steady respectable young man, was a..."
3,Jane Austen,To him therefore the succession to the Norland...
4,Jane Austen,"Their mother had nothing, and their father onl..."


In [6]:
# Guardamos el dataset como un archivo CSV
df.to_csv('data/classifier/sentences.csv', index=False)

In [7]:
# Contar el número de datos de entrenamiento por cada autor
author_counts = df['author'].value_counts()

# Convertir los conteos en un DataFrame
summary_df = author_counts.reset_index()
summary_df.columns = ['author', 'num_training_data']

summary_df

,author,num_training_data
0,Leo Tolstoy,10514
1,Jane Austen,3745
2,James Joyce,2815


## 2. Preprocesamiento

Preprocesamos el dataset separandolo en entrenamiento y prueba. Adicionalmente, tokenizamos el texto para poder mapear las palabras a los embeddings construidos y usarlos como la capa de entrada de los modelos de redes neuronales.

In [8]:
# Conjunto de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(df['sentence'], df['author'],
                                                    train_size=0.7, random_state=42)

In [9]:
# Tokenizacion y preprocesamiento
def preprocess_text(text: str) -> list[str]:
    """
    Limpia y tokeniza el texto mediante:
    1. Eliminación de puntuación y caracteres especiales.
    2. Convierte el texto a minúsculas.
    3. Tokenización del texto en palabras.
    4. Eliminación de palabras vacías (stopwords).
    
    Args:
    text (str): Texto de entrada a preprocesar.
    
    Returns:
    list: Una lista de tokens (palabras) del texto limpiado.
    """
    # Eliminar cualquier carácter no alfabético, números, etc.
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenizar y convertir el texto a minúsculas
    tokens = gensim.utils.simple_preprocess(text, deacc=True)
    
    # Eliminar palabras vacías (stopwords)
    tokens = [word for word in tokens if word not in STOPWORDS]
    
    return tokens

x_train = x_train.apply(preprocess_text)
x_train.head()

6165    [wine, gluttony, idleness, laziness, irritabil...
7583    [old, stateliness, cleanliness, stillness, rei...
1052    [letters, town, days, nerve, elinors, body, th...
4038    [soft, road, phaeton, pursuing, tenour, way, b...
3405    [weston, walked, purpose, tired, sit, time, re...
Name: sentence, dtype: object

In [10]:
# Mapeamos las palabras a los embeddings de texto pre-entrenados
# TODO: Cargar el modelo de embeddings de texto